In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

In [2]:
def get_states_tables():
    """
    Goes through the website 'http://www.wahlrecht.de/umfragen/laender.htm'
    and extracts the table for states individually, 
    
    Return: a dictionary containing the id names of the states as keywords and the 
            pd dataframes as values.
    """
    tables = {} # {'state': df}

    page = urllib.request.urlopen('http://www.wahlrecht.de/umfragen/laender.htm')
    soup = BeautifulSoup(page, 'html.parser')
    
    # Find the subtables
    states = soup.find_all('th', colspan='10', id=True)
    rows = soup.find_all('tr')
    header = [col.get_text() for col in soup.find_all('th', class_=True, limit=9)]

    # Initialize with empty/unimportant values
    table = [] # df
    name = "ignore"
    for row in rows:
        # Start point of a new state
        if row.find('th', colspan='10', id=True) != None:
            table = []
            name = row.contents[1].get('id')
            #header = row.find_all_next('th', class_=True, limit=9)

        # Read the data of the subtable
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        table.append([ele for ele in cols if ele])
        #table.append([ele if ele else None for ele in cols])

        # End point for each state
        if row.find('th', colspan='10', class_="trenner") != None: 
            # Don't use the information outside the states.
            if name != "ignore" and name != 'hb':
                tables[name] = pd.DataFrame(table, columns=header)
            # Handle differentely the information from hb = Bremen, which is empty.
            elif name == 'hb':
                tables[name] = pd.DataFrame(table, columns=header[:7])
    # Add last table, that doesn't have trenner at the end
    tables[name] = pd.DataFrame(table, columns=header)

    return tables

In [3]:
states = get_states_tables()

In [4]:
states.keys()

dict_keys(['bb', 'nw', 'sn', 'ni', 'he', 'by', 'st', 'hh', 'rp', 'hb', 'be', 'mv', 'bw', 'sh', 'sl', 'th'])

In [5]:
table = states['be']

# drop rows containing only NaNs/None
table = table.dropna(axis=0, how='all')

# replace all empty entries with NaN
table = table.replace('', 'NaN', regex=True)

table.index = np.arange(table.shape[0])

# drop the rows containing the true results of the elections
Idx = np.where(table['Institut(Datum)'].str.contains('Bundestagswahl'))
table = table.drop(Idx[0])
table.index = np.arange(table.shape[0])

# split 'BefrateZeitraum' into two columns
for i, n in enumerate(table['BefragteZeitraum']):
    if len(n) > 13:
        n = n.split('\n', 1)[0]
        table['BefragteZeitraum'][i] = n[:-13]
table.rename(columns={'BefragteZeitraum': 'Befragte'}, inplace=True)

# split the column 'Institut(Datum)' into two columnbs
institut_datum = table['Institut(Datum)'].str.extract('([A-z]+)?([(])?(\d+.\d+.\d+)')
institut_datum = institut_datum.drop(1, axis=1)
institut_datum.columns = ['Institut', 'Datum']
table = pd.concat([institut_datum, table.iloc[:,1:]], axis=1)

# convert the date to type date
table.Datum = pd.to_datetime(table.Datum).dt.date

C:\Users\jiame\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [6]:
# replace the strings %,-
table = table.replace(',', '.', regex=True)
table = table.replace('[–?%)≈/]', '', regex=True)

In [7]:
table

,Institut,Datum,Auftrag-geber,Befragte,CDU,SPD,GRÜNE,FDP,LINKE,Sonstige
0,Forsa,2017-05-30,BerlinerZeitung,1.001,29,24,10,8,15,AfD 7 Sonst. 7
1,Infratestdimap,2017-05-23,RBB. BerlinerMorgenpost,1.000,29,22,11,8,16,AfD 10 Sonst. 4
2,Forsa,2017-03-05,BerlinerZeitung,1.002,25,28,9,6,16,AfD 8 Sonst. 8
3,Forsa,2017-03-04,BerlinerZeitung,1.005,25,28,10,6,15,AfD 8 Sonst. 8
4,Forsa,2017-02-27,BerlinerZeitung,1.004,21,30,10,6,15,AfD 10 Sonst. 8
5,Forsa,2017-01-30,BerlinerZeitung,1.008,26,20,14,6,15,AfD 13 Sonst. 6
6,Forsa,2017-02-01,BerlinerZeitung,1.003,25,21,14,5,16,AfD 14 Sonst. 5
7,Forsa,2016-11-27,BerlinerZeitung,1.001,25,21,14,5,16,AfD 14 Sonst. 5
8,Forsa,2016-10-30,BerlinerZeitung,1.002,24,21,15,5,17,AfD 12 Sonst. 6
9,Forsa,2016-07-31,BerlinerZeitung,1.002,24,21,17,6,17,AfD 9 Sonst. 6


In [232]:
"""
code from wahlrecht_polling_firms that is left to do for states
"""
# replace all empty entries with NaN
table = table.replace('', 'NaN', regex=True)

# if the colomn Sonstige contains entries with more than one number
try: 
    table.Sonstige = table.Sonstige.astype(float)
except ValueError:
    for i, n in enumerate(table.Sonstige):
        if len(n) > 2:
            digits = np.array([digit for digit in np.arange(10).astype(str) if digit in n])
            table.Sonstige[i] = digits.astype(int).sum()
            table.Sonstige = table.Sonstige.astype(float)

# convert all numbers to float
table[table.keys()[1:]] = table[table.keys()[1:]].astype(float)

TypeError: object of type 'numpy.int32' has no len()

TODO: 
- catch the shifts in a row due to missing values
- adjust the box above for states